In [ ]:
!pip install openpyxl

In [6]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Utility to remove non-printable characters
def clean_excel_string(val):
    if isinstance(val, str):
        # Removes ASCII control characters (0-31)
        return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f]', '', val)
    return val

# --- 1. IMPORT & PREPROCESS ---
# Load dataset
df = pd.read_csv('full.csv')
df_raw_count = df.isna().sum() 

# Drop unnecessary columns (area_unit and scraper) and rows missing critical data
cols_to_drop = ['area_unit', 'scraper']
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')
df = df.dropna(subset=['price', 'property_type', 'area'])

# Business Logic: Set floor count to 1 for Apartments/Condos if left empty
apt_types = ['Căn hộ chung cư', 'Căn hộ chung cư mini', 'Condotel']
if 'n_floors' in df.columns:
    df.loc[(df['property_type'].isin(apt_types)) & (df['n_floors'].isna()), 'n_floors'] = 1

# --- 2. MACHINE LEARNING IMPUTATION LOGIC ---
def smart_imputer(df_input, target_col):
    if target_col not in df_input.columns:
        return df_input[target_col] if target_col in df_input.columns else None
    
    working_df = df_input.copy()
    # Features used to learn the patterns
    features = ['property_type', 'price', 'area', 'city_province', 'district']
    features = [f for f in features if f in working_df.columns]
    
    # Encode categorical features for mathematical processing
    for col in features:
        if working_df[col].dtype == 'object':
            working_df[col] = LabelEncoder().fit_transform(working_df[col].astype(str))

    train_set = working_df[working_df[target_col].notna()]
    predict_set = working_df[working_df[target_col].isna()]
    
    if len(predict_set) > 0 and len(train_set) > 0:
        # CASE A: Numeric Data (Floors, Width, Bedrooms, etc.) -> Use Regressor
        if df_input[target_col].dtype in [np.float64, np.int64]:
            model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
            model.fit(train_set[features], train_set[target_col])
            preds = model.predict(predict_set[features])
            
            if 'n_' in target_col: # Round results for count-based columns
                preds = np.round(preds)
            df_input.loc[df_input[target_col].isna(), target_col] = preds
            
        # CASE B: Text/Categorical Data (Interior) -> Use Classifier
        else:
            model = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
            le_target = LabelEncoder()
            y_train = le_target.fit_transform(train_set[target_col].astype(str))
            
            model.fit(train_set[features], y_train)
            y_pred = model.predict(predict_set[features])
            # Decode numbers back to original text (e.g., "Full Furniture", "Basic")
            df_input.loc[df_input[target_col].isna(), target_col] = le_target.inverse_transform(y_pred)
    
    return df_input[target_col]

# --- 3. EXECUTE ML PROCESSING ---
print("--- STARTING ML IMPUTATION FOR MISSING VALUES ---")
targets = ['n_bedrooms', 'n_bathrooms', 'n_floors', 'front_width', 'front_road_width', 'interior']
for target in targets:
    if target in df.columns:
        print(f"  > Imputing: {target}")
        df[target] = smart_imputer(df, target)

# --- 4. GEOSPATIAL COORDINATES PROCESSING (AVERAGING) ---
print("--- CALCULATING COORDINATE AVERAGES BY REGION ---")
if 'latitude' in df.columns and 'longitude' in df.columns:
    # Step 1: Fill NaN using the mean of the same District
    df['latitude'] = df['latitude'].fillna(df.groupby(['city_province', 'district'])['latitude'].transform('mean'))
    df['longitude'] = df['longitude'].fillna(df.groupby(['city_province', 'district'])['longitude'].transform('mean'))
    
    # Step 2: For districts with zero coordinate data, fallback to the Province average
    df['latitude'] = df['latitude'].fillna(df.groupby('city_province')['latitude'].transform('mean'))
    df['longitude'] = df['longitude'].fillna(df.groupby('city_province')['longitude'].transform('mean'))

# --- 5. FINAL DATA FORMATTING ---
# Convert count-based columns to Integer type
for col in ['n_bedrooms', 'n_bathrooms', 'n_floors']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).round().astype(int)

# Apply character cleaning across the entire dataframe
df = df.map(clean_excel_string)

# --- 6. METRICS REPORT & EXPORT ---
print("\n" + "="*75)
print(f"{'DATA FIELD':<20} | {'ORIGINAL NaN':<12} | {'FINAL NaN':<10} | {'RECOVERY RATE'}")
print("-" * 75)

nan_after = df.isna().sum()
for col in df_raw_count.index:
    if col in df.columns:
        before = df_raw_count[col]
        after = nan_after[col]
        recovered = before - after
        pct = f"{(recovered/before*100):.1f}%" if before > 0 else "100%"
        print(f"{col:<20} | {before:<12} | {after:<10} | {pct}")

# --- 7. EXPORT TO CSV ---
output_filename = 'full_cleaned.csv'
# utf-8-sig ensures Vietnamese characters are readable in Excel CSV
df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("="*75)
print(f"SUCCESS! Cleaned data saved to: {output_filename}")

--- STARTING ML IMPUTATION FOR MISSING VALUES ---
  > Imputing: n_bedrooms
  > Imputing: n_bathrooms
  > Imputing: n_floors
  > Imputing: front_width
  > Imputing: front_road_width
  > Imputing: interior
--- CALCULATING COORDINATE AVERAGES BY REGION ---

DATA FIELD           | ORIGINAL NaN | FINAL NaN  | RECOVERY RATE
---------------------------------------------------------------------------
Unnamed: 0           | 0            | 0          | 100%
property_type        | 0            | 0          | 100%
price                | 1367         | 0          | 100.0%
area                 | 0            | 0          | 100%
n_bedrooms           | 4218         | 0          | 100.0%
n_bathrooms          | 5117         | 0          | 100.0%
n_floors             | 8194         | 0          | 100.0%
address              | 3            | 3          | 0.0%
city_province        | 0            | 0          | 100%
district             | 0            | 0          | 100%
legal_docs           | 1875         